<a href="https://colab.research.google.com/github/LeonardoGoncRibeiro/01_DataScienceUsingPython/blob/main/08_SpellChecker_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing: Implementing a portuguese spell checker

**These are class notes from the course in NLP from Alura: "Corretor Ortográfico em Python: aplicando técnicas de NLP". Here, we will show how to build a portuguese spell checker using NLP concepts.**

Machines do not "talk" the same language as humans. When using machines to solve a given problem related to language, one needs to use an intermediate to process natural language. This intermediate is denominated as **Natural Language Prossessing (NLP)**. Using NLP, for instance, one may implement a personal assistant (such as Amazon's Alexa) or a sentiment analysis. NLP allows us to perform man-machine communication!

In this file, we will use techniques from Natural Language Processing (NLP) to build a portuguese spell checker. 

Our algorithm will receive an incorrectly typed word and will then generate (or propose) the right word. This spell checker will be built based on a database of possible words. This is known as the **training set**. 

## Understanding our problem

**Problem:** People often type text incorrectly. How can we make a good educated guess on what people really meant to type (in portuguese)?

For instance, when someone writes

*lgica*

We should be able to guess that people actually wanted to say:

*lógica*




# Importing our database

The first thing we have to do is to import a database, which will serve as our training set. It is through this database that our machine will be able to "learn" our language and, then, make good suggestions on what people actually meant to say.

On NLP, the database is also known as "corpus". Here, as our "corpus", we will use a .txt file with different articles (documents) from alura.com.br

Let's import this data. To that end, we will open "artigos.txt" and store the text characters in a variable.

In [3]:
with open("artigos.txt", "r") as f:
  articles = f.read( )

Let's look at the first 500 characters:

In [4]:
print(articles[:500])




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:

java 

Suponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou


Is this corpus interesting for our spell checker? Does it has a high number of words? A high number of distinct words?

## Checking the number of words in our corpus

To check the number of words, we first have to group each word. After all, if we just use



```
# len(articles)
```

we are not assessing the number of words, but rather the number of characters.


In [5]:
len(articles)

2605046

In [6]:
len("Hi")

2

The group that forms each word is denominated as a token. To create a token, we can use:

In [7]:
text_example = "Hi, are you okay?"

tokens_example = text_example.split( )
tokens_example

['Hi,', 'are', 'you', 'okay?']

Then, we can use *len( )* to count the number of words:

In [8]:
len(tokens_example)

4

Note that, in our tokens, there are commas and punctuation marks. We can improve our tokenization by removing those. We do this using the nltk package.

First, let's import and download some minor nltk add-ons

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Now, let's perform our tokenization.

In [10]:
tokens_example = nltk.tokenize.word_tokenize(text_example)

In [11]:
tokens_example

['Hi', ',', 'are', 'you', 'okay', '?']

Now we have our words correctly separated! However, note that it captured commas and punctuation marks as a separated token. However, we only need the words! 

To separate punctuation from actual words, we can create an auxiliary function:

In [12]:
def GetWords(token_list):
  words_list = []
  for token in token_list:
    if token.isalpha( ):
      words_list.append(token)
  return words_list

Note that, here, we are using the *isalpha( )* function to help in defining if a token is alphabetical. Now, applying to our example token:

In [13]:
GetWords(tokens_example)

['Hi', 'are', 'you', 'okay']

Great! Now everything worked out fine.

Let's apply the same idea to our corpus:

In [14]:
tokens_raw = nltk.tokenize.word_tokenize(articles)
print(tokens_raw[:50])

['imagem', 'Temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', ':', 'java', 'Para', 'salvar', 'um', 'novo', 'usuário', ',', 'várias', 'validações', 'são', 'feitas', ',', 'como', 'por', 'exemplo', ':', 'Ver', 'se', 'o', 'nome', 'só', 'contém', 'letras', ',', '[', '**o', 'CPF', 'só', 'números**', ']', 'e', 'ver', 'se', 'o', 'usuário', 'possui', 'no']


Note that, in our list of tokens, there are commas and punctuation marks. Now using our user-defined function:

In [15]:
tokens = GetWords(tokens_raw)
print(tokens[:50])

['imagem', 'Temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', 'java', 'Para', 'salvar', 'um', 'novo', 'usuário', 'várias', 'validações', 'são', 'feitas', 'como', 'por', 'exemplo', 'Ver', 'se', 'o', 'nome', 'só', 'contém', 'letras', 'CPF', 'só', 'e', 'ver', 'se', 'o', 'usuário', 'possui', 'no', 'mínimo', 'anos', 'Veja', 'o', 'método', 'que', 'faz', 'essa', 'validação']


Now, everything looks great. Let's get the number of words (or tokens) in our corpus:

In [16]:
num_words = len(tokens)

Thus, our corpus has 393,914 words! 

## Checking the number of unique words in our corpus

Even though we have 393,914 words, we should always keep in mind that we have a lot of repeated entries. Also, note that, if we have these two words:

*   Bread
*   bread

They are actually the same word, although with different capitalization. First, it is import to "normalize" all of our words. For that end, we can define another auxiliary function



In [17]:
def NormalizeTokens(word_list):
  word_list_lower = []
  for word in word_list:
    word_list_lower.append(word.lower( ))

  return word_list_lower

Then, applying to our list of tokens:

In [18]:
print(tokens[:10])

['imagem', 'Temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no']


In [19]:
tokens_norm = NormalizeTokens(tokens)

In [20]:
print(tokens_norm[:10])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no']


Now, all of our tokens are in lowercase. Now, let's keep only our unique words. Thus, we have to remove our repeated words.

Python has a very interesting idea, known as *sets*. Using *set( )* on a list, it passes the unique values from that list. 

In [21]:
print(set(tokens_norm))

{'exclusivo', 'mudarmos', 'avisos', 'olhado', 'compreensivo', 'pixar', 'pereira', 'redimensionamento', 'cuidado', 'picos', 'realizem', 'berkeley', 'update', 'composta', 'livrar', 'extende', 'manteremos', 'fascinante', 'interesso', 'monospace', 'critical', 'opcional', 'simularemos', 'escaneáveis', 'tentamos', 'acertado', 'maiuscula', 'undefined', 'conectar', 'pagariam', 'absorvido', 'hashcode', 'width', 'fita', 'uma', 'importantíssimo', 'recarregadas', 'percam', 'pais', 'mobilegeddon', 'workana', 'conhecida', 'consigamos', 'construir', 'encontrassem', 'saga', 'aproximamos', 'canudo', 'jsdom', 'pegar', 'sofisticadas', 'interligação', 'fôlego', 'atual', 'solte', 'pois', 'encryption', 'compradas', 'unidos', 'memorizado', 'conveniência', 'funcionamento', 'veicular', 'restaurantes', 'tentará', 'aproximadamente', 'resultaram', 'exigindo', 'tr', 'snapchat', 'disparar', 'criatividade', 'períodos', 'piada', 'complementam', 'corpo', 'resumidos', 'quit', 'truncar', 'maximize', 'produzido', 'reaçõe

This variable, however, has the type "set". Thus, we may simply do:

In [22]:
tokens = list(set(tokens_norm))
print(tokens[:50])

['exclusivo', 'mudarmos', 'avisos', 'olhado', 'compreensivo', 'pixar', 'pereira', 'redimensionamento', 'cuidado', 'picos', 'realizem', 'berkeley', 'update', 'composta', 'livrar', 'extende', 'manteremos', 'fascinante', 'interesso', 'monospace', 'critical', 'opcional', 'simularemos', 'escaneáveis', 'tentamos', 'acertado', 'maiuscula', 'undefined', 'conectar', 'pagariam', 'absorvido', 'hashcode', 'width', 'fita', 'uma', 'importantíssimo', 'recarregadas', 'percam', 'pais', 'mobilegeddon', 'workana', 'conhecida', 'consigamos', 'construir', 'encontrassem', 'saga', 'aproximamos', 'canudo', 'jsdom', 'pegar']


Finally, we may correctly evaluate the number of non-repeated entries:

In [23]:
len(tokens)

17652

Great! Our corpus has more than 17,000 unique entries. 

# Developing and testing our spell checker

Now that we have our corpus, we may start developing and testing our spell checker. First, we will make a basic checker which:

*   Checks if there is any missing letter; and
*   Suggests a right word to replace.

To make this, we can test inserting in our word any letter in any part of the word. For instance, if we pass *lgica* to our algorithm, the algorithm should test various candidates until finding out that the correct word is *lógica*.

For that end, our algorithm should test different cases, joining two "sides" of the word and putting a letter between them. Our algorithm should test different words, such as:

*   ólgica
*   lógica
*   lgóica
*   lgióca
*   lgicóa
*   lgicaó

Of course, we also don't know if "ó" is the right missing letter. Thus, we have to make this for every possible letter.



## Adding letters

Let's start to work on the code. First, let's define a function to generate candidate solutions for our algorithm:

In [24]:
def GetSliceList(word):                  # This function will generate the list of possible slices
  slices = []                     
  for i in range(len(word) + 1):         # In this loop we will create a list of pairs of slices (left and right slices)
    slice_l = word[:i]                   # Left slice
    slice_r = word[i:]                   # Right slice
    slices.append( (slice_l, slice_r) )  # Storing the slices as a tuple on a list
  return slices

In [25]:
def CandidateSolutions(word):            # This function will generate candidate solutions

  slices = GetSliceList(word)            # This function will get of list of tuples, each indicating a possible slice
  cand_sols = InsertLetters(slices)      # This function will insert letters in our slices, creating our candidate solutions

  return cand_sols


In [26]:
def InsertLetters(slices):
  letters = 'abcdefghijklmnopqrstuvwxyzáâàãéêèíîìóôòúûù'  # We will insert these letters
  candidate_solutions = []              
  for left, right in slices:                              # We will make a loop over the list of tuples 
    for letter in letters:                                # We will make a loop over the list of letters
      candidate_solutions.append(left + letter + right)   # We will join the left and right slices, putting the letter between them

  return candidate_solutions

Now let's see if our algorithm is working:

In [27]:
cand_sol = CandidateSolutions('lgica')
print(cand_sol)

['algica', 'blgica', 'clgica', 'dlgica', 'elgica', 'flgica', 'glgica', 'hlgica', 'ilgica', 'jlgica', 'klgica', 'llgica', 'mlgica', 'nlgica', 'olgica', 'plgica', 'qlgica', 'rlgica', 'slgica', 'tlgica', 'ulgica', 'vlgica', 'wlgica', 'xlgica', 'ylgica', 'zlgica', 'álgica', 'âlgica', 'àlgica', 'ãlgica', 'élgica', 'êlgica', 'èlgica', 'ílgica', 'îlgica', 'ìlgica', 'ólgica', 'ôlgica', 'òlgica', 'úlgica', 'ûlgica', 'ùlgica', 'lagica', 'lbgica', 'lcgica', 'ldgica', 'legica', 'lfgica', 'lggica', 'lhgica', 'ligica', 'ljgica', 'lkgica', 'llgica', 'lmgica', 'lngica', 'logica', 'lpgica', 'lqgica', 'lrgica', 'lsgica', 'ltgica', 'lugica', 'lvgica', 'lwgica', 'lxgica', 'lygica', 'lzgica', 'lágica', 'lâgica', 'làgica', 'lãgica', 'légica', 'lêgica', 'lègica', 'lígica', 'lîgica', 'lìgica', 'lógica', 'lôgica', 'lògica', 'lúgica', 'lûgica', 'lùgica', 'lgaica', 'lgbica', 'lgcica', 'lgdica', 'lgeica', 'lgfica', 'lggica', 'lghica', 'lgiica', 'lgjica', 'lgkica', 'lglica', 'lgmica', 'lgnica', 'lgoica', 'lgpica',

It does seem to be working fine! We can even see that our correct word is in the list

In [28]:
cand_sol[78]

'lógica'

However, how do we find the right word automatically? After all, that is exactly what a spell checker should do.

An idea is to evaluate the likelihood that a given word can be the right guess. Then, to find the best possible guess, we simply choose the word with the maximum likelihood!

To evaluate this likelihood, we can simply get a candidate solution and see how many times this candidate solution appears in our corpus. We may use a function from nltk:

In [29]:
freq = nltk.FreqDist(tokens_norm)

Let's see our most common words:

In [30]:
freq.most_common(20)

[('de', 15494),
 ('o', 13966),
 ('que', 12225),
 ('a', 11034),
 ('e', 10478),
 ('para', 7694),
 ('um', 6346),
 ('é', 5881),
 ('uma', 5202),
 ('do', 5116),
 ('em', 4718),
 ('com', 4711),
 ('como', 3809),
 ('no', 3354),
 ('da', 3282),
 ('os', 3153),
 ('mais', 3039),
 ('não', 3037),
 ('se', 3021),
 ('por', 2693)]

Everything seems to be working out fine.

Our checker should be able to assess the frequency of each word and, then, get the candidate solution with the highest frequency. So, we will first create a function to evaluate the likelihood of a given word, given by the frequency of that word on our corpus divided by the number of words.

In [31]:
def likelihood(word, freq, num_words):
  return freq[word]/num_words

Let's test our function. If we try to get the likelihood of 'lagica':

In [32]:
likelihood('lagica', freq, num_words)

0.0

We get 0.0, because there is no such word in our corpus. However, if we try to get the likelihood of 'lógica':

In [33]:
likelihood('lógica', freq, num_words)

0.00022086039084673304

Everything seems to be working out fine. Now, we will create our spell checker:

In [34]:
def spell_checker(word, freq, num_words):
  cand_sols = CandidateSolutions(word)       # Getting all candidate soltions
  max_like, right_word = 0, None             # Creating the maximum likelihood and the right word variables
  for sol in cand_sols:                      # Initiating loops through candidate solutions
    like = likelihood(sol, freq, num_words)  # Evaluating likelihood of a candidate solution
    if like > max_like:                      # Checking if its likelihood is higher than the maximum likelihood
      right_word, max_like = sol, like       # If so, the right_word and the max_like variables are updated    

  # These lines of code are commented so that our testing does not get overcrowded.
  
  # if right_word == None:
  #  print("The spell checker was not able to find a good educated guess for the correct word.")
  
  return right_word

Good! Let's test our checker:

In [35]:
spell_checker("lgica", freq, num_words)

'lógica'

Great! The checker was able to return the right word. Let's try with other words:

In [36]:
spell_checker("armáro", freq, num_words)

'armário'

In [37]:
spell_checker("piza", freq, num_words)

'pizza'

In [38]:
spell_checker("lógice", freq, num_words)

Note that, this time, our spell checker only works for one type of error: when people forget to use a single letter. However, when people use a wrong letter instead of the right one, or when people use an additional letter, our checker does not work! 

Let's see how to fix other types of errors.

# Checking the quality of our spell checker

Before fixing other types of errors, let's use a test set. A test set is a very good tool to test an algorithm, so that we can compare two different approaches. 

In this case, our test set is composed of multiple words which need to be corrected by the use of a spell checker. Let's see how many words our current checker is able to fix, and then let's determine the reliability of our checker. Here, the reliability is given by:

* Number of time our checker suggested the right word divided by the number of words tested.

Let's create a function to evaluate the reliability of our checker:

In [39]:
def CheckerReliability(testset, freq, num_words, func_checker):
  hit = 0
  for right_word, test_word in testset:
    guess = func_checker(test_word, freq, num_words) # Note that we are passing our checker function as an argument! 
    if guess == right_word:
      hit += 1

  reliab = hit/len(testset)

  print(f"Reliability: {round(reliab*100,2)}% (number of words tested: {len(testset)})")

Now, let's create a function to read the test set and store it in a list.

In [40]:
def GetTestData(name_file):
  test_words = []
  f = open(name_file, "r")
  for row in f:
    right, wrong = row.split( )
    test_words.append( (right, wrong) )
  f.close( )
  return test_words

Note that our test set needs to have two informations: one for the **right word**, which is the word our checker needs to find as the output, and one the the **misspelled word**, which is passed to the checker.

Finally, let's use our functions to check the reliability of our checker.

In [41]:
testset = GetTestData('palavras.txt')

CheckerReliability(testset, freq, num_words, spell_checker)

Reliability: 1.08% (number of words tested: 186)


Note that our spell checker was able to find the correct guess only 1.08% of the time.

Ok, so let's try to improve our spell checker.

# Improving our spell checker

Now let's give an added functionality to our checker. Now, it should be able to check if there is any **addtional** letter in the word and delete it, generating the correct word. For instance, it must be able to receive a misspelled word, such as


*   *lóigica*

and guess the correct word:

*   *lógica*





Note that, now, our spell checker is not able to solve such a problem:

In [42]:
spell_checker("lóigica", freq, num_words)

So, let's start improving our spell checker. 

## Deleting letters

Once again, we will break our word in two slices. To that end, we can use our already defined function:

In [43]:
test_word = 'lóigica'

print(GetSliceList(test_word))

[('', 'lóigica'), ('l', 'óigica'), ('ló', 'igica'), ('lói', 'gica'), ('lóig', 'ica'), ('lóigi', 'ca'), ('lóigic', 'a'), ('lóigica', '')]


Now, let's create a new function to create candidate solutions, this time by removing one letter from our word:

In [44]:
def CandidateSolutions2(word):           # This function will generate candidate solutions

  slices = GetSliceList(word)            # This function will get of list of tuples, each indicating a possible slice
  cand_sols = RemoveLetters(slices)      # This function will remove letters in our slices, creating our candidate solutions

  return cand_sols


And finally, let's implement our letter deletion algorithm:

In [45]:
def RemoveLetters(slices):
  candidate_solutions = []              
  for left, right in slices:                     # We will make a loop over the list of tuples 
    candidate_solutions.append(left + right[1:]) # We will join the left and right slices, removing the first letter from the right slice

  return candidate_solutions

Now, let's test if our candidate solutions make sense:

In [46]:
CandidateSolutions2('lóigica')

['óigica',
 'ligica',
 'lógica',
 'lóiica',
 'lóigca',
 'lóigia',
 'lóigic',
 'lóigica']

Great! We can even see our right word (*lógica*) at index 2 in our list of candidate solutions. 

Now, let's redefine our spell checker function.

In [47]:
def spell_checker2(word, freq, num_words):
  cand_sols1 = CandidateSolutions(word)      # Getting candidate solutions for the letter addition operation
  cand_sols2 = CandidateSolutions2(word)     # Getting candidate solutions for the letter deletion operation
  cand_sols = [*cand_sols1, *cand_sols2]     # This will concatenate both lists in a single one
  max_like, right_word = 0, None             # Creating the maximum likelihood and the right word variables
  for sol in cand_sols:                      # Initiating loops through candidate solutions
    like = likelihood(sol, freq, num_words)  # Evaluating likelihood of a candidate solution
    if like > max_like:                      # Checking if its likelihood is higher than the maximum likelihood
      right_word, max_like = sol, like       # If so, the right_word and the max_like variables are updated    

  # These lines of code are commented so that our testing does not get overcrowded.
  
  # if right_word == None:
  #  print("The spell checker was not able to find a good educated guess for the correct word.")
  
  return right_word

Note that, instead of having to redefine our spell checker function, we could have just redefined our function to generate candidate solutions, so that it could give us all possible solutions at once. 

Let's just test if our new checker is able to guess the correct word for our example:


In [48]:
test_word = 'lóigica'

spell_checker2(test_word, freq, num_words)

'lógica'

Nice! Finally, let's test again our spell checkers. For the first checker, we got a reliability of only 1.08%:

In [49]:
CheckerReliability(testset, freq, num_words, spell_checker)

Reliability: 1.08% (number of words tested: 186)


Now, for the second checker:

In [50]:
CheckerReliability(testset, freq, num_words, spell_checker2)

Reliability: 41.4% (number of words tested: 186)


Great! We were able to improve our reliability by adding a new functionality to our checker. Now it is able to show the correct guess for 41.4% of the tested words.

By adding more functionalities we may be able to improve this reliability even more.

# Improving our spell checker even more

## Changing letters

Our checker is already able to solve two kinds of problems:

*   Adding a letter;
*   Deleting a letter.

Now, let's add a new functionality, where the checker will be able to:

*   Change letters.

For instance, when receiving the word *lígica*, the checker should be able to guess the word *lógica* by testing candidate solutions generated via a change in the letters of the word.





So, let's start by creating **another** function to generate candidate solutions:

In [51]:
def CandidateSolutions3(word):           # This function will generate candidate solutions

  slices = GetSliceList(word)            # This function will get of list of tuples, each indicating a possible slice
  cand_sols = ChangeLetters(slices)      # This function will change letters in our slices, creating our candidate solutions

  return cand_sols

And let's implement our function to change letters. Note that this operation is basically a sum between the two others: we will delete a letter and then add a new letter in its place. So, the function *ChangeLetters( )* can be defined as:



In [52]:
def ChangeLetters(slices):
  letters = 'abcdefghijklmnopqrstuvwxyzáâàãéêèíîìóôòúûù'     # We will insert these letters
  candidate_solutions = []              
  for left, right in slices:                                 # We will make a loop over the list of tuples 
    for letter in letters:                                   # We will make a loop over the list of letters
    
      candidate_solutions.append(left + letter + right[1:])  # We will join the left and right slices, removing the first 
                                                             # letter from the right slice and adding a new letter

  return candidate_solutions

So, let's test our candidate solutions:

In [53]:
cand3 = CandidateSolutions3('lígica')
print(cand3)

['aígica', 'bígica', 'cígica', 'dígica', 'eígica', 'fígica', 'gígica', 'hígica', 'iígica', 'jígica', 'kígica', 'lígica', 'mígica', 'nígica', 'oígica', 'pígica', 'qígica', 'rígica', 'sígica', 'tígica', 'uígica', 'vígica', 'wígica', 'xígica', 'yígica', 'zígica', 'áígica', 'âígica', 'àígica', 'ãígica', 'éígica', 'êígica', 'èígica', 'íígica', 'îígica', 'ìígica', 'óígica', 'ôígica', 'òígica', 'úígica', 'ûígica', 'ùígica', 'lagica', 'lbgica', 'lcgica', 'ldgica', 'legica', 'lfgica', 'lggica', 'lhgica', 'ligica', 'ljgica', 'lkgica', 'llgica', 'lmgica', 'lngica', 'logica', 'lpgica', 'lqgica', 'lrgica', 'lsgica', 'ltgica', 'lugica', 'lvgica', 'lwgica', 'lxgica', 'lygica', 'lzgica', 'lágica', 'lâgica', 'làgica', 'lãgica', 'légica', 'lêgica', 'lègica', 'lígica', 'lîgica', 'lìgica', 'lógica', 'lôgica', 'lògica', 'lúgica', 'lûgica', 'lùgica', 'líaica', 'líbica', 'lícica', 'lídica', 'líeica', 'lífica', 'lígica', 'líhica', 'líiica', 'líjica', 'líkica', 'lílica', 'límica', 'línica', 'líoica', 'lípica',

Great! Everything seems to be working out fine. We can even find our correct guess in the list.

In [54]:
cand3[78]

'lógica'

Now, we just have to implement the third iteration of our checker:

In [55]:
def spell_checker3(word, freq, num_words):
  cand_sols1 = CandidateSolutions(word)                   # Getting candidate solutions for the letter addition operation
  cand_sols2 = CandidateSolutions2(word)                  # Getting candidate solutions for the letter deletion operation
  cand_sols3 = CandidateSolutions3(word)                  # Getting candidate solutions for the letter changing operation

  cand_sols = [*cand_sols1, *cand_sols2, *cand_sols3]     # This will concatenate both lists in a single one
  max_like, right_word = 0, None                          # Creating the maximum likelihood and the right word variables
  for sol in cand_sols:                                   # Initiating loops through candidate solutions
    like = likelihood(sol, freq, num_words)               # Evaluating likelihood of a candidate solution
    if like > max_like:                                   # Checking if its likelihood is higher than the maximum likelihood
      right_word, max_like = sol, like                    # If so, the right_word and the max_like variables are updated    

  # These lines of code are commented so that our testing does not get overcrowded.
  
  # if right_word == None:
  #  print("The spell checker was not able to find a good educated guess for the correct word.")
  
  return right_word

Good. Now, let's test the reliability of our checkers once again:

In [56]:
CheckerReliability(testset, freq, num_words, spell_checker)

Reliability: 1.08% (number of words tested: 186)


In [57]:
CheckerReliability(testset, freq, num_words, spell_checker2)

Reliability: 41.4% (number of words tested: 186)


In [58]:
CheckerReliability(testset, freq, num_words, spell_checker3)

Reliability: 76.34% (number of words tested: 186)


Great! Now our checker is able to guess the correct word in 76.34% of cases in our test set! Still, there is one more type of error that we may want to fix.

## Inverting letters

Sometimes, we just invert our letters. For instance, when trying to spell *lógica*, it is common to write, for instance, *lóigca*, especially when typing fast.

To do so, we may simply get our slices, and then invert the first and second letters from our right slices.

So, let's implement an algorithm that will invert those letters, and generate new candidate solutions.

In [59]:
def InvertLetters(slices):
  candidate_solutions = []              
  for left, right in slices:                                            # We will make a loop over the list of tuples 

    if (len(right) < 2):                                                # First, we will test if our right slice has at least 2 letters
      break                                                             # If not, the loop will break

    candidate_solutions.append(left + right[1] + right[0] + right[2:])  # We will join the left and right slices, but we will invert 
                                                                        # the first and second letters from the right slice

  return candidate_solutions

Now, implementing our new function to evaluate candidate solutions:

In [60]:
def CandidateSolutions4(word):           # This function will generate candidate solutions

  slices = GetSliceList(word)            # This function will get of list of tuples, each indicating a possible slice
  cand_sols = InvertLetters(slices)      # This function will invert letters in our slices, creating our candidate solutions

  return cand_sols

Ok. Let's test our new function:

In [61]:
test_word = 'lóigca'

cand = CandidateSolutions4(test_word)
print(cand)

['óligca', 'liógca', 'lógica', 'lóicga', 'lóigac']


Great! Again, we can even see our correct guess, at index 2.

Finally, implementing our fourth and final checker:

In [62]:
def spell_checker4(word, freq, num_words):
  cand_sols1 = CandidateSolutions(word)                            # Getting candidate solutions for the letter addition operation
  cand_sols2 = CandidateSolutions2(word)                           # Getting candidate solutions for the letter deletion operation
  cand_sols3 = CandidateSolutions3(word)                           # Getting candidate solutions for the letter changing operation
  cand_sols4 = CandidateSolutions4(word)                           # Getting candidate solutions for the letter invertion operation

  cand_sols = [*cand_sols1, *cand_sols2, *cand_sols3, *cand_sols4] # This will concatenate both lists in a single one
  max_like, right_word = 0, None                                   # Creating the maximum likelihood and the right word variables
  for sol in cand_sols:                                            # Initiating loops through candidate solutions
    like = likelihood(sol, freq, num_words)                        # Evaluating likelihood of a candidate solution
    if like > max_like:                                            # Checking if its likelihood is higher than the maximum likelihood
      right_word, max_like = sol, like                             # If so, the right_word and the max_like variables are updated    

  # These lines of code are commented so that our testing does not get overcrowded.
  
  # if right_word == None:
  #  print("The spell checker was not able to find a good educated guess for the correct word.")
  
  return right_word

Testing our checker:

In [63]:
spell_checker4(test_word, freq, num_words)

'lógica'

Good! Now, let's assess the reliability of our checkers once again:

In [64]:
CheckerReliability(testset, freq, num_words, spell_checker)

Reliability: 1.08% (number of words tested: 186)


In [65]:
CheckerReliability(testset, freq, num_words, spell_checker2)

Reliability: 41.4% (number of words tested: 186)


In [66]:
CheckerReliability(testset, freq, num_words, spell_checker3)

Reliability: 76.34% (number of words tested: 186)


In [67]:
CheckerReliability(testset, freq, num_words, spell_checker4)

Reliability: 76.34% (number of words tested: 186)


Note that our new implementation was not able to improving upon our reliability. This means that it was not able to improve upon our previous iteration. Note that the reliability as also a function of our test set: Most likely our test set had no misspelled words with this kind of error.

Now, let's see how much time each of checkers take to run. For that end, we will run *CheckerReliability( )* one time for each checker, and see how much time each checker takes.

In [68]:
%time CheckerReliability(testset, freq, num_words, spell_checker)

Reliability: 1.08% (number of words tested: 186)
CPU times: user 34.5 ms, sys: 0 ns, total: 34.5 ms
Wall time: 35.4 ms


In [69]:
%time CheckerReliability(testset, freq, num_words, spell_checker2)

Reliability: 41.4% (number of words tested: 186)
CPU times: user 38.9 ms, sys: 0 ns, total: 38.9 ms
Wall time: 41.2 ms


In [70]:
%time CheckerReliability(testset, freq, num_words, spell_checker3)

Reliability: 76.34% (number of words tested: 186)
CPU times: user 71.4 ms, sys: 0 ns, total: 71.4 ms
Wall time: 72 ms


In [71]:
%time CheckerReliability(testset, freq, num_words, spell_checker4)

Reliability: 76.34% (number of words tested: 186)
CPU times: user 75 ms, sys: 857 µs, total: 75.8 ms
Wall time: 77 ms


Note that there is a very small loss of efficiency as we improve our checker. Note that this time spent is not fixed, and different runs of our algorithm could get different results. Let's reimplement our *CheckerReliability( )* function and then do a similar test, now checking how much time it takes to run the algorithm **100** times. 

In [72]:
def CheckerReliability_NoPrint(testset, freq, num_words, func_checker):
  hit = 0
  for right_word, test_word in testset:
    guess = func_checker(test_word, freq, num_words) # Note that we are passing our checker function as an argument! 
    if guess == right_word:
      hit += 1

  reliab = hit/len(testset)

In [73]:
%time for _ in range(100): CheckerReliability_NoPrint(testset, freq, num_words, spell_checker)

CPU times: user 3.14 s, sys: 12.1 ms, total: 3.15 s
Wall time: 3.16 s


In [74]:
%time for _ in range(100): CheckerReliability_NoPrint(testset, freq, num_words, spell_checker2)

CPU times: user 3.29 s, sys: 10.8 ms, total: 3.31 s
Wall time: 3.32 s


In [75]:
%time for _ in range(100): CheckerReliability_NoPrint(testset, freq, num_words, spell_checker3)

CPU times: user 6.76 s, sys: 18 ms, total: 6.78 s
Wall time: 6.8 s


In [76]:
%time for _ in range(100): CheckerReliability_NoPrint(testset, freq, num_words, spell_checker4)

CPU times: user 6.89 s, sys: 19.6 ms, total: 6.91 s
Wall time: 6.93 s


Now, we can see more clearly that, at each iteration, our checker becomes more and more inefficient. So, as we add new operations to the checker, we have to keep in mind that these operations take **time**, and may turn our checker into a very costly algorithm.

## Discussing the checker error

Ok, we managed to create a checker with a very good reliability of 76.34%. Also, we saw that, as we incremented our checker with new functionalities, it became **more reliable**, but also **less efficient**. 

However, it is important to note that this error is not entirely due to the algorithm: it is also related to our **vocabulary**. If our checker does not know the right word, it is not able to make a good guess.

So, let's see if our checker knows all right words from our test set.

In [77]:
def RateKnownWords(dataset, testset):
  num_known_words = 0
  for right, wrong in testset:
    if right in dataset:
      num_known_words += 1
  print(f'Of the {len(testset)} words in our testset, our corpus knows {num_known_words} words ({round(num_known_words/len(testset)*100 ,2)}%).')

Now, let's run this function using the right parameters

In [78]:
RateKnownWords(tokens, testset)

Of the 186 words in our testset, our corpus knows 173 words (93.01%).


So, there are 13 words in our test set that our algorithm does not know!

# Improving our spell checker even further

Ok, our checker is able to guess the right words for multiple types of errors, generating candidate solutions via the following operations:

*   Letter addition
*   Letter deletion
*   Letter changing
*   Letter invertion

However, it only guess the correct word when **only one** operation is necessary. For instance, if we try to check 'lóggica':





In [79]:
spell_checker4('lóggica', freq, num_words)

'lógica'

we are able to find the correct guess, using the letter deletion operation. However, if we check 'lóggicca':

In [80]:
spell_checker4('lóggicca', freq, num_words)

we are not able to find the correct guess, as we actually need to perform the letter deletion operation **twice**.

So, let's make another iteration of our checker. This time the checker will be able to employ two operations, generating **candidate solutions for our candidate solutions**. First, let's implement an unique function to get our candidate solutions:

In [90]:
def GetAllCandidateSols(word):
  cand_sols1 = CandidateSolutions(word)  # Getting candidate solutions for the letter addition operation
  cand_sols2 = CandidateSolutions2(word) # Getting candidate solutions for the letter deletion operation
  cand_sols3 = CandidateSolutions3(word) # Getting candidate solutions for the letter changing operation
  cand_sols4 = CandidateSolutions4(word) # Getting candidate solutions for the letter invertion operation

  cand_sols = [*cand_sols1, *cand_sols2, *cand_sols3, *cand_sols4]

  return cand_sols

Now, let's implement a new iteration of our checker:

In [91]:
def spell_checker5(word, freq, num_words):
  cand_sols_it1 = GetAllCandidateSols(word)                        # This function will get all candidate solutions for our misspelled word

  cand_sols_it2 = cand_sols_it1                                    # In this variable, we will store the candidate solutions for our candidates
  for cand_sol in cand_sols_it1:
    new_cands = GetAllCandidateSols(cand_sol)                      # Getting the candidate solutions for each former candidate
    cand_sols_it2 = [*cand_sols_it2, *new_cands]                   # Storing all candidates in a single list

  max_like, right_word = 0, None                                   # Creating the maximum likelihood and the right word variables
  for sol in cand_sols_it2:                                        # Initiating loops through candidate solutions
    like = likelihood(sol, freq, num_words)                        # Evaluating likelihood of a candidate solution
    if like > max_like:                                            # Checking if its likelihood is higher than the maximum likelihood
      right_word, max_like = sol, like                             # If so, the right_word and the max_like variables are updated    

  # These lines of code are commented so that our testing does not get overcrowded.
  
  # if right_word == None:
  #  print("The spell checker was not able to find a good educated guess for the correct word.")
  
  return right_word

Great. Now, let's test with our example word.

In [92]:
spell_checker5('lóggicca', freq, num_words)

'lógica'

It worked! However, is this checker efficient? Let's see how much time it takes to fix only one word:

In [93]:
%time spell_checker5('lóggicca', freq, num_words)

CPU times: user 5.42 s, sys: 2.55 s, total: 7.97 s
Wall time: 7.98 s


'lógica'

Note that this checker is **very** inefficient. Is there any way to improve the efficiency of our checker?

First, let's look at the number of candidate solutions. For our previous algorithm, we had 772 candidate solutions:



In [87]:
cand_sols_it1 = GetAllCandidateSols('lóiigica') 
len(cand_sols_it1)

772

However, for our new checker, we have 632,188 candidate solutions:

In [94]:
  cand_sols_it1 = GetAllCandidateSols('lóiigica')                      

  cand_sols_it2 = cand_sols_it1                                    
  for cand_sol in cand_sols_it1:
    new_cands = GetAllCandidateSols(cand_sol)                     
    cand_sols_it2 = [*cand_sols_it2, *new_cands]     

len(cand_sols_it2)             

632188

This way, we have to perform way to many checks too finally define our right guess!

Let's try to perform a more efficient checking process. First, we can remove our duplicate candidate solutions. Then, we can check if our candidate is in our corpus. If not, we can simply remove it, and then we don't need to check its likelihood. Let's implement these ideas:

In [95]:
def spell_checker6(word, freq, num_words):
  cand_sols_it1 = GetAllCandidateSols(word)                        # This function will get all candidate solutions for our misspelled word

  cand_sols_it2 = cand_sols_it1                                    # In this variable, we will store the candidate solutions for our candidates
  for cand_sol in cand_sols_it1:
    new_cands = GetAllCandidateSols(cand_sol)                      # Getting the candidate solutions for each former candidate
    cand_sols_it2 = [*cand_sols_it2, *new_cands]                   # Storing all candidates in a single list

  cand_sols_it3 = set(cand_sols_it2)                               # Removing duplicates

  cand_sols_it4 = []
  for cand_sol in cand_sols_it3:
    if cand_sol in freq.keys( ):                                   # Checking if our candidate solution is in our vocabulary
      cand_sols_it4.append(cand_sol)

  max_like, right_word = 0, None                                   # Creating the maximum likelihood and the right word variables
  for sol in cand_sols_it4:                                        # Initiating loops through candidate solutions
    like = likelihood(sol, freq, num_words)                        # Evaluating likelihood of a candidate solution
    if like > max_like:                                            # Checking if its likelihood is higher than the maximum likelihood
      right_word, max_like = sol, like                             # If so, the right_word and the max_like variables are updated    

  # These lines of code are commented so that our testing does not get overcrowded.
  
  # if right_word == None:
  #  print("The spell checker was not able to find a good educated guess for the correct word.")
  
  return right_word

In [96]:
spell_checker6('lóiigica', freq, num_words)

'lógica'

Great! Our checker works! 

Now, let's check our time spent.

In [97]:
%time spell_checker6('lóggicca', freq, num_words)

CPU times: user 5.41 s, sys: 951 ms, total: 6.36 s
Wall time: 6.38 s


'lógica'

Note that our checker is slightly more efficient than the previous one. Now, let's test our checker reliability:

In [98]:
%time CheckerReliability(testset, freq, num_words, spell_checker6)

Reliability: 55.38% (number of words tested: 186)
CPU times: user 7min 26s, sys: 2min, total: 9min 26s
Wall time: 9min 28s


Wait, why did our checker reliability lowered?

That occurred because our new checker is receiving a misspelled word and correcting it to a wrong word. For instance, take the case where one writes 'esje' instead of 'esse'. Let's use our checker:

In [99]:
spell_checker6('esje', freq, num_words)

'se'

We actually get the word 'se', and not 'este', as expected. That occurs because the likelihood of 'se' is much higher than that of 'este':

In [101]:
likelihood('se', freq, num_words)

0.007669186675264144

In [102]:
likelihood('este', freq, num_words)

0.0005508816645257594

This behavior is very common when we allow a model to be very general. We say that these models have a very low **bias**, but a very large **variance**. That means that our new model is very exposed to overfitting our data.

Thus, actually, our best checker was *spell_checker4( )*, which showed a good reliability and efficiency:

In [103]:
%time spell_checker4('lógicca', freq, num_words)

CPU times: user 1.12 ms, sys: 0 ns, total: 1.12 ms
Wall time: 1.27 ms


'lógica'